# Smart Unlocking System with Face Recognition

# Import required Libraries

In [1]:
import serial
import urllib.request
import numpy as np
import cv2
import time
from deepface import DeepFace
from imutils import face_utils
import face_recognition
import dlib
from scipy.spatial import distance as dist


## Train Face encodings for face recognition

In [2]:
uday_image = face_recognition.load_image_file("train/uday.jpg")
uday_face_encoding = face_recognition.face_encodings(uday_image)[0]
uday_image2 = face_recognition.load_image_file("train/uday2.jpg")
uday2_face_encoding = face_recognition.face_encodings(uday_image2)[0]
uday_image1 = face_recognition.load_image_file("train/uday1.jpg")
uday1_face_encoding = face_recognition.face_encodings(uday_image1)[0]
uday_image3 = face_recognition.load_image_file("train/uday3.jpg")
uday3_face_encoding = face_recognition.face_encodings(uday_image3)[0]
uday_image4 = face_recognition.load_image_file("train/uday4.jpg")
uday4_face_encoding = face_recognition.face_encodings(uday_image4)[0]
sai_image = face_recognition.load_image_file("train/sai_chandra.jpg")
sai_face_encoding = face_recognition.face_encodings(sai_image)[0]
sai_image1 = face_recognition.load_image_file("train/sai_chandra1.jpg")
sai1_face_encoding = face_recognition.face_encodings(sai_image1)[0]
sai_image2 = face_recognition.load_image_file("train/sai_chandra2.jpg")
sai2_face_encoding = face_recognition.face_encodings(sai_image2)[0]
sai_image3 = face_recognition.load_image_file("train/sai_chandra3.jpg")
sai3_face_encoding = face_recognition.face_encodings(sai_image3)[0]


known_face_encodings = [
    uday_face_encoding,
    uday1_face_encoding,
    uday2_face_encoding,
    uday3_face_encoding,
    uday4_face_encoding,
    sai_face_encoding,
    sai1_face_encoding,
    sai2_face_encoding,
    sai3_face_encoding
]
known_face_names = [
    "Udaya Sankar",
    "Udaya Sankar",
    "Udaya Sankar",
    "Udaya Sankar",
    "Udaya Sankar",
    "Sai Chandra",
    "Sai Chandra",
    "Sai Chandra",
    "Sai Chandra"
]

In [3]:
url = 'http://192.168.0.17/capture'

# Show Time! 

In [ ]:


face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')



first_read = True


#video_capture = cv2.VideoCapture(0)

BREAK = False
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

EYE_AR_THRESH = 0.3
EYE_AR_CONSEC_FRAMES = 3


COUNTER = 0
TOTAL = 0

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

#MAX_BUFF_LEN = 255

ser = serial.Serial()
ser.baudrate = 115200
ser.port = 'COM3'
ser.open()
var = 'a'
c = var.encode()
var1 = 'b'
d = var1.encode()
#video_capture = cv2.VideoCapture(0)
STOP = False
Fail = False
while True:
    if STOP:
        break
   #start = time.time()
    while True:
        if STOP:
            break
        #current = time.time()
        #if current - start > 15:
        #    Fail = True
        #    break
        imgresp = urllib.request.urlopen(url)
        imgnp = np.array(bytearray(imgresp.read()),dtype=np.uint8)
        img =cv2.imdecode(imgnp,-1)

        #img = cv2.resize(img, (0, 0), fx=0.25, fy=0.25)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        rects = detector(gray, 0)

        for rect in rects:

            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)

            leftEye = shape[lStart:lEnd]
            rightEye = shape[rStart:rEnd]
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)

            ear = (leftEAR + rightEAR) / 2.0

            leftEyeHull = cv2.convexHull(leftEye)
            rightEyeHull = cv2.convexHull(rightEye)
            cv2.drawContours(img, [leftEyeHull], -1, (0, 255, 0), 1)
            cv2.drawContours(img, [rightEyeHull], -1, (0, 255, 0), 1)

            if ear < EYE_AR_THRESH:
                COUNTER += 1

            else:

                if COUNTER >= EYE_AR_CONSEC_FRAMES:
                    TOTAL += 1

                COUNTER = 0
            cv2.putText(img, "Blink your eyes!",(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)

            if TOTAL >=1:
                TOTAL = 0
                BREAK = True

        cv2.imshow("img", img)
        #key = cv2.waitKey(1)
        
        if BREAK:
            BREAK = False
            break
        if cv2.waitKey(1) & 0xFF == ord('q'):
            STOP = True
            break
    if STOP:
        break
    #if Fail:
    #    Fail = False
    #    continue
        
    start = time.time()
    
    while True:
        current = time.time()
        if current - start > 15:
            Fail = True
            break
        imgresp = urllib.request.urlopen(url)
        imgnp = np.array(bytearray(imgresp.read()),dtype=np.uint8)
        img =cv2.imdecode(imgnp,-1)
        result = DeepFace.analyze(img_path = img, actions = ['emotion'], enforce_detection=False)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray,1.1,4)
        for (x,y,w,h) in faces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),3)

        emotion = result["dominant_emotion"]

        txt = str(emotion)

        cv2.putText(img,"Blink detected Now smile",(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)

        cv2.imshow('img',img)
        if txt == "happy":
            break
        if cv2.waitKey(1) & 0xFF == ord('q'):
            STOP = True
            break
        
    face_locations = []
    face_encodings = []
    face_names = []
    process_this_frame = True
    
    if Fail:
        Fail = False
        continue
    
    if STOP:
        break
    
    start = time.time()    
    while True:
        current = time.time()
        if current - start > 15:
            Fail = True
            break
        
        if BREAK:
            break

        imgresp = urllib.request.urlopen(url)
        imgnp = np.array(bytearray(imgresp.read()),dtype=np.uint8)
        img =cv2.imdecode(imgnp,-1)

        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(img, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = small_frame[:, :, ::-1]

        # Only process every other frame of video to save time
        if process_this_frame:
            # Find all the faces and face encodings in the current frame of video
            face_locations = face_recognition.face_locations(rgb_small_frame)
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

            face_names = []
            for face_encoding in face_encodings:
                # See if the face is a match for the known face(s)
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                name = "Unknown"

                # # If a match was found in known_face_encodings, just use the first one.
                # if True in matches:
                #     first_match_index = matches.index(True)
                #     name = known_face_names[first_match_index]

                # Or instead, use the known face with the smallest distance to the new face
                face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name = known_face_names[best_match_index]

                face_names.append(name)

        process_this_frame = not process_this_frame


        # Display the results
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            # Scale back up face locations since the frame we detected in was scaled to 1/4 size
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            # Draw a box around the face
            cv2.rectangle(img, (left, top), (right, bottom), (0, 0, 255), 2)

            # Draw a label with a name below the face
            cv2.rectangle(img, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(img, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        # Display the resulting image
        cv2.imshow('img', img)
        if "Sai Chandra" in face_names:
            ser.write(c)
            time.sleep(5)
            break
        elif "Udaya Sankar" in face_names:
            ser.write(d)
            time.sleep(5)
            break

        # Hit 'q' on the keyboard to quit!
        if cv2.waitKey(1) & 0xFF == ord('q'):
            STOP = True
            break
    if Fail:
        Fail = False
        continue
    if STOP:
        break

    

#video_capture.release()
cv2.destroyAllWindows()

# THANKYOU
### By
### Kandukuru Udaya Sankar,
### Janga Sai Chandra